In [1]:
import numpy as np
import pandas as pd
from numba import njit
from my_stuff import MufexKeys, BinanceUSDMTestKeys, ApexTestKeys
from quantfreedom.enums import *
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.indicators.indicators import *
from quantfreedom.plotting.plot_funcs import plot_candles_1_ind_dif_pane, plot_candles_1_ind_same_pane
from quantfreedom.exchanges.binance_exchange.binance_usdm import BinanceUSDM
from quantfreedom.exchanges.apex_exchange.apex import Apex


from quantfreedom.exchanges.exchange import Exchange
np.set_printoptions(formatter={"float_kind": "{:0.1f}".format})
from my_stuff import MufexTestKeys


mufex_test = Mufex(api_key=MufexTestKeys.api_key, 
                   secret_key=MufexTestKeys.secret_key, 
                   use_test_net=True,
                   )

mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,use_test_net=False,
    
)

%load_ext autoreload
%autoreload 2

In [ ]:
candles = mufex_main.get_candles(
    symbol="BTCUSDT",
    timeframe="5m",
    candles_to_dl=200,
)
open = candles[:1]
high = candles[:, 2]
low = candles[:, 3]
close = candles[:, 4]


close_shift = np.roll(candles[:, 4], 1)
candles

In [ ]:
mufex_test.create_order(
    params={
        "symbol": "BTCUSDT",
        "positionIdx": 1,
        "side": "Buy",
        "orderType": "Market",
        "qty": str(0.01),
        "timeInForce": "ImmediateOrCancel",
    }
)

In [2]:
apex_test = Apex(
    api_key=ApexTestKeys.api_key,
    secret_key=ApexTestKeys.secret_key,
    passphrase=ApexTestKeys.passphrase,
    stark_key_private=ApexTestKeys.stark_key_private,
    stark_key_public=ApexTestKeys.stark_key_public,
    stark_key_y=ApexTestKeys.stark_key_y,
    use_test_net=True
)

In [3]:
apex_test.create_order(
    symbol="BTC-USDC",
    side="BUY",
    type="LIMIT",
    size=.005,
    price=34575,
    # expirationEpochSeconds=currentTime,
    # limitFeeRate=limitFeeRate,
)

TypeError: conversion from NoneType to Decimal is not supported

In [ ]:
binance_test = BinanceUSDM(
    api_key=BinanceUSDMTestKeys.api_key,
    secret_key=BinanceUSDMTestKeys.secret_key,
    use_test_net=True,
)

In [ ]:
binance_test.create_order(symbol="BTCUSDT", side="BUY", type="Limit")